In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.constraints import MaxNorm

from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import GridSearchCV
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
import joblib
from scipy import stats

In [2]:
train = pd.read_excel('processed_train.xlsx',index_col=0)
train.head()

,admission number,Age,Sex,Follow_up time,Revasc,Culp,MainVASc,Numsten,ECG,Killip,...,UA,Glu,WBC,N,HyperP,Cigaret,OtherHis,diabetes,Drugcom,MACE
342,1038735,49,0,433,3260,4,1,1,1,1,...,455.0,4.13,9.28,7.06,0,1,0,0,1,0
31,1000681,63,0,646,251,6,4,2,1,4,...,307.0,9.77,10.91,8.12,1,1,0,0,1,0
2,1035609,68,1,581,270,2,1,1,1,3,...,406.0,5.01,4.60,2.75,1,0,1,0,0,1
77,1027073,80,0,497,398,5,0,1,0,1,...,445.0,5.77,6.33,4.85,0,0,1,0,1,0
371,1050791,35,0,377,132,2,2,3,1,1,...,402.0,5.89,13.36,12.04,1,0,1,0,1,0


In [3]:
X = train.drop(['admission number','MACE'],axis=1)
y = train['MACE']

X=(X-np.min(X))/(np.max(X)-np.min(X))

In [4]:
#hyperparameter tune of DT
DT_model = DecisionTreeClassifier(class_weight='balanced',random_state=1,max_depth=6,
                                 max_features=5,criterion='gini',splitter='best')
'''
parameters = {'max_depth':range(2,20),'max_features':['log2','sqrt',1,2,3,4,5,6,7,8,9,10,11,12,13,14],
              'criterion':['gini','entropy'],'splitter':['best','random'],}
step1:parameter1 = {'max_depth':range(2,20)}，best_params_：{'max_depth': 6}
step2:parameter2 = {'max_features':['log2','sqrt',1,2,3,4,5,6,7,8,9,10,11,12,13,14]}，best_params_：{{'max_features': 5}}
step3:parameter3 = {'criterion':['gini','entropy']}，best_params_：{'criterion': 'gini'}
step4:parameter4 = {'splitter':['best','random']}，best_params_：{'splitter': 'best'}
step5:parameter5 = {'min_samples_leaf':range(1,10),'min_samples_split':range(2,10)}，
best_params_：{'min_samples_leaf': 6, 'min_samples_split': 2}

Thus, the best model: DecisionTreeClassifier(class_weight='balanced',random_state=1,
                                  max_depth=6,max_features=5,criterion='gini',
                                 splitter='best',min_samples_leaf=6,min_samples_split=2)
'''
parameter5 = {'min_samples_leaf':range(1,10),'min_samples_split':range(2,10)}
GS = GridSearchCV(DT_model, parameter5,scoring='roc_auc', cv=5)
model = GS.fit(X,y)

print(model.best_params_)
'''
tuned_DT_model =  DecisionTreeClassifier(class_weight='balanced',random_state=1,
                                  max_depth=6,max_features=5,criterion='gini',
                                 splitter='best',min_samples_leaf=6,min_samples_split=2)
'''
tuned_DT_model = model.best_estimator_
joblib.dump(tuned_DT_model,filename='best_DT_model')
print(model.best_score_)

{'min_samples_leaf': 6, 'min_samples_split': 2}
0.6640106951871658


In [5]:
#calibration for decision tree
isotonic = CalibratedClassifierCV(model.best_estimator_, cv=5, method='isotonic')
calibrate_DT = isotonic.fit(X,y)
joblib.dump(calibrate_DT,filename='calibrate_DT')

['calibrate_DT']

In [6]:
#caculate AUC in the internal validation cohort
parameter = {'min_samples_leaf':[6],'min_samples_split':[2]}
GS1 = GridSearchCV(DT_model, parameter,scoring='roc_auc', cv=5)
model1 = GS1.fit(X,y)
result = model1.cv_results_
auc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
#caculate 95% CI of AUC
interval1=stats.norm.interval(0.95,np.mean(auc_result),np.std(auc_result))
print('AUC:',auc_result)
print('Mean AUC:',np.mean(auc_result))
print('95% CI of AUC',interval1)

#caculate accuracy in the internal validation cohort
GS2 = GridSearchCV(DT_model, parameter,scoring='accuracy', cv=5)
model2 = GS2.fit(X,y)
result = model2.cv_results_
acc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval2=stats.norm.interval(0.95,np.mean(acc_result),np.std(acc_result))
print('Acc',acc_result)
print('Mean Acc',np.mean(acc_result))
print('95% CI of AUC',interval2)

#caculate accuracy in the internal validation cohort
GS3 = GridSearchCV(DT_model, parameter,scoring='f1', cv=5)
model3 = GS3.fit(X,y)
result = model3.cv_results_
f1_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval3=stats.norm.interval(0.95,np.mean(f1_result),np.std(f1_result))
print('F1_score',f1_result)
print('Mean F1',np.mean(f1_result))
print('95% CI of F1',interval3)


AUC: [0.57941176 0.70686275 0.64411765 0.5745098  0.81515152]
Mean AUC: 0.6640106951871658
95% CI of AUC (0.4881745767221536, 0.8398468136521781)
Acc [0.57142857 0.69387755 0.67346939 0.51020408 0.77083333]
Mean Acc 0.6439625850340136
95% CI of AUC (0.46302730759099175, 0.8248978624770354)
F1_score [0.46153846 0.59459459 0.52941176 0.42857143 0.64516129]
Mean F1 0.5318555079465894
95% CI of F1 (0.3740998263955928, 0.689611189497586)


In [7]:
#hyperparameter tune of LR
LR_model = LogisticRegression(class_weight="balanced",penalty='l2',random_state=1,
                              dual=False,C=1,solver='lbfgs')
'''
parameters ={'penalty':['l1','l2'],'dual'[True,False],'solver':['liblinear','lbfgs','newton-cg','sag','saga'],
'C':[0.01,0.05,0.1,0.5,1,5,10,50],'fit_intercept':[True,False],'intercept_scaling':[-0.5,0.5,1,1.5,2]}

if penalty = l1
step1:parameter1 = {'penalty':['l1'],'dual':[False],'solver':['liblinear','saga']} 
best_params_:{'dual': False, 'penalty': 'l1', 'solver': 'liblinear'}
step2:parameter2 = {'C':[0.01,0.05,0.1,0.5,1,5,10,50]} best_params_: {'C': 5} best_score_:0.7127
step3:parameter3 = {'fit_intercept':[True],'intercept_scaling':[0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.9,0.95,1]} 
best_params_ = {'fit_intercept': True, 'intercept_scaling': 0.75} best_score_:0.7131
when penalty = l1,tuned_LR_model =  LogisticRegression(class_weight="balanced",random_state=1,penalty='l1',dual=False,
                                    solver='liblinear',C=5,fit_intercept=True,intercept_scaling=0.75)

if penalty = l2,dual = True,solver = liblinear
step1:parameter1 = {'penalty':['l2],'dual':[True],'solver':['liblinear'],'C':[0.01,0.05,0.1,0.5,1,5,10,50]}
best_params_: {'C': 1, 'dual': True, 'penalty': 'l2', 'solver': 'liblinear'} best_score_:0.7168
step2:parameter2 = {'fit_intercept':[True],'intercept_scaling':[0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.9,0.95,1]} 
best_params_:{'fit_intercept': True, 'intercept_scaling': 0.9} best_score_:0.7168
Thus,tuned_LR_model = LogisticRegression(class_weight="balanced",penalty='l2',random_state=1,
                              dual=True,solver='liblinear',C=1,fit_intercept=False)

if penalty = l2,dual = False
step1:parameter1 = {'penalty':['l2],'dual':[False],'solver':['liblinear','lbfgs','newton-cg','saga']}
best_params_:{'C': 1, 'dual': False, 'penalty': 'l2', 'solver': 'lbfgs'} best_score_:0.7173
step2:parameter2 = {'fit_intercept':[True],'intercept_scaling':[0.5,0.75,1,1.25,1.5]} 
best_params_:{'fit_intercept': True, 'intercept_scaling': 0.5}  best_score_:0.7173 
Thus,the best tuned LR model is:best_LR_model = LogisticRegression(class_weight="balanced",penalty='l2',random_state=1,
                              dual=False,C=1,solver='lbfgs',fit_intercept=True,intercept_scaling=0.5)
                              
Above all,the best tuned LR model is:LogisticRegression(class_weight="balanced",penalty='l2',random_state=1,
                                                        dual=False,C=1,solver='lbfgs',fit_intercept=True,intercept_scaling=0.5
'''
parameter2 = {'fit_intercept':[True],'intercept_scaling':[0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.9,0.95,1]}
GS = GridSearchCV(LR_model, parameter2,scoring='roc_auc', cv=5)
model = GS.fit(X,y)
'''
tuned_LR_model = LogisticRegression(class_weight="balanced",penalty='l2',random_state=1,
                              solver='liblinear',C=0.25,fit_intercept=True,intercept_scaling=1)
'''
tuned_LR_model = model.best_estimator_
joblib.dump(tuned_LR_model,filename='best_LR_model')
print(model.best_params_)
print(model.best_score_)

{'fit_intercept': True, 'intercept_scaling': 0.5}
0.7172727272727274


In [8]:
#calibration for logistic regression 
isotonic = CalibratedClassifierCV(model.best_estimator_, cv=5, method='isotonic')
calibrate_LR = isotonic.fit(X,y)
joblib.dump(calibrate_LR,filename='calibrate_LR')

['calibrate_LR']

In [9]:
#LR
#caculate AUC in the internal validation cohort
parameter = {'fit_intercept':[True],'intercept_scaling':[0.5]}
GS1 = GridSearchCV(LR_model, parameter,scoring='roc_auc', cv=5)
model1 = GS1.fit(X,y)
result = model1.cv_results_
auc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
#caculate 95% CI of AUC
interval1=stats.norm.interval(0.95,np.mean(auc_result),np.std(auc_result))
print('AUC:',auc_result)
print('Mean AUC:',np.mean(auc_result))
print('95% CI of AUC',interval1)

#caculate accuracy in the internal validation cohort
GS2 = GridSearchCV(LR_model, parameter,scoring='accuracy', cv=5)
model2 = GS2.fit(X,y)
result = model2.cv_results_
acc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval2=stats.norm.interval(0.95,np.mean(acc_result),np.std(acc_result))
print('Acc',acc_result)
print('Mean Acc',np.mean(acc_result))
print('95% CI of AUC',interval2)

#caculate accuracy in the internal validation cohort
GS3 = GridSearchCV(LR_model, parameter,scoring='f1', cv=5)
model3 = GS3.fit(X,y)
result = model3.cv_results_
f1_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval3=stats.norm.interval(0.95,np.mean(f1_result),np.std(f1_result))
print('F1_score',f1_result)
print('Mean F1',np.mean(f1_result))
print('95% CI of F1',interval3)


AUC: [0.70196078 0.73529412 0.7254902  0.72058824 0.7030303 ]
Mean AUC: 0.7172727272727274
95% CI of AUC (0.6918597380572815, 0.7426857164881733)
Acc [0.69387755 0.73469388 0.7755102  0.73469388 0.66666667]
Mean Acc 0.7210884353741496
95% CI of AUC (0.6475748595503642, 0.7946020111979349)
F1_score [0.51612903 0.60606061 0.62068966 0.55172414 0.52941176]
Mean F1 0.5648030392256002
95% CI of F1 (0.4834314356827193, 0.646174642768481)


In [10]:
#GaussianNB has no parameters that can be optimized, therefore we just perform 5_fold crossvalidation,
# then caculate the AUC in each split validation cohort
X1 = X.copy()
X1.index = range(len(X))
y1 = y.copy()
y1.index = range(len(y))

k = 5
num_val_samples = len(X1) // k
validation_result = []
Auc = []
Acc = []
F1 = []
#5_Fold cross_validation
for i in range(k):
    val_data = X1[i * num_val_samples: (i+1) * num_val_samples]
    val_targrt = y1[i * num_val_samples: (i+1) * num_val_samples]
    
    train_data = np.concatenate([X1[:i * num_val_samples],
                               X1[(i+1) * num_val_samples:]],axis = 0)
    train_targrt = np.concatenate([y1[:i * num_val_samples],
                                   y1[(i+1) * num_val_samples:]],axis = 0)
    NB_model = GaussianNB()
    NB_model = NB_model.fit(train_data,train_targrt)
    y_proba = NB_model.predict_proba(val_data)
    y_pred = NB_model.predict(val_data)

    Auc.append(metrics.roc_auc_score(val_targrt,y_proba[:,1]))
    Acc.append(metrics.accuracy_score(val_targrt,y_pred))
    F1.append(metrics.f1_score(val_targrt,y_pred))
tuned_NB_model = NB_model
joblib.dump(tuned_NB_model,filename='best_NB_model')

['best_NB_model']

In [11]:
#calibration for Naive Bayes
isotonic = CalibratedClassifierCV(GaussianNB(), cv=5, method='isotonic')
calibrate_NB = isotonic.fit(X,y)
joblib.dump(calibrate_NB,filename='calibrate_NB')

['calibrate_NB']

In [12]:
#NB
#caculate AUC in the internal validation cohort
#caculate 95% CI of AUC
interval1=stats.norm.interval(0.95,np.mean(Auc),np.std(Auc))
print('AUC:',Auc)
print('Mean AUC:',np.mean(Auc))
print('95% CI of AUC',interval1)

#caculate accuracy in the internal validation cohort
interval2=stats.norm.interval(0.95,np.mean(Acc),np.std(Acc))
print('Acc',Acc)
print('Mean Acc',np.mean(Acc))
print('95% CI of AUC',interval2)

#caculate accuracy in the internal validation cohort
interval3=stats.norm.interval(0.95,np.mean(F1),np.std(F1))
print('F1_score',F1)
print('Mean F1',np.mean(F1))
print('95% CI of F1',interval3)


AUC: [0.73109243697479, 0.6875, 0.6851851851851851, 0.7849364791288566, 0.7777777777777778]
Mean AUC: 0.7332983758133219
95% CI of AUC (0.6498575391245583, 0.8167392125020855)
Acc [0.7083333333333334, 0.7083333333333334, 0.75, 0.75, 0.7916666666666666]
Mean Acc 0.7416666666666667
95% CI of AUC (0.6805540523286235, 0.8027792810047099)
F1_score [0.4615384615384615, 0.4166666666666667, 0.45454545454545453, 0.6, 0.5833333333333334]
Mean F1 0.5032167832167833
95% CI of F1 (0.3581726785360275, 0.6482608878975391)


In [13]:
Linear_SVM_model = LinearSVC(class_weight='balanced',random_state=1,penalty='l1',dual=False,
                            fit_intercept=False,C=0.5)
'''
parameters = {'penalty':['l2','l1'],'dual':[True,False],'loss':['squared_hinge','hinge'],'fit_intercept':[True,False],
             'C':[0.001,0.005,0.1,0.5,1,5,10],'intercept_scaling':[-0.5,0.5,1,1.5,2]}
penalty = l2
step1:parameter1 = {'penalty':['l2'],'dual':[True,False]} best_params_:{'dual': True, 'penalty': 'l2'}
step2:parameter2 = {'fit_intercept':[True,False]} best_params_: {'fit_intercept': True} best_score_:0.7099
step3:parameter3 = {'fit_intercept':[True],'intercept_scaling':[0.5,1,1.5,2]} best_params_:{'fit_intercept': True, 'intercept_scaling': 1}
best_score_:0.7084. Thus,best_params_:{'fit_intercept': False}
step4:parameter4 = {'C':[0.001,0.005,0.01,0.05,0.1,0.5,1,5,10]} best_params_:{'C': 0.1} best_score_:0.7168
step5:parameter5 = {'loss':['squared_hinge','hinge']}  best_params_: {'loss': 'squared_hinge'} best_score_:0.7168
if penalty = l2, Best model =  LinearSVC(class_weight='balanced',random_state=1,penalty = 'l2',dual=True,
                            fit_intercept=False,C=0.1), best_score_:0.7168
                            
penalty = l1
step1:parameter1 = {'penalty':['l1'],'dual':[False]} best_params_:{'dual': False, 'penalty': 'l1'} best_score_:0.709
step2:parameter2 = {'fit_intercept':[True,False]} best_params_: {'fit_intercept': False} best_score_:0.7119
step3:parameter3 = {'C':[0.001,0.005,0.01,0.05,0.1,0.5,1,5,10]}   best_params_:{'C': 0.5} best_score_:0.7133
step4:parameter4 = {'loss':['squared_hinge','hinge']} best_params_:{'loss': 'squared_hinge'},best_score_:0.7288

Above all, Best_LinearSVC_Model = LinearSVC(class_weight='balanced',random_state=1,penalty = 'l2',dual=True,
                                  fit_intercept=False,C=0.1)
'''
parameter5 = {'loss':['squared_hinge']}
GS = GridSearchCV(Linear_SVM_model, parameter5,scoring='roc_auc', cv=5)
model = GS.fit(X,y)
'''
Best_LinearSVC_Model = LinearSVC(class_weight='balanced',random_state=1,penalty = 'l2',dual=True,
                                 fit_intercept=False,C=0.025,loss='squared_hinge')
'''
Tuned_LinearSVC_Model = model.best_estimator_
joblib.dump(Tuned_LinearSVC_Model,filename='Best_LinearSVC_Model')
print(model.best_params_)
print(model.best_score_)

{'loss': 'squared_hinge'}
0.7133986928104574


In [14]:
#calibration for LinearSVC
isotonic = CalibratedClassifierCV(model.best_estimator_, cv=5, method='isotonic')
calibrate_LSVC = isotonic.fit(X,y)
joblib.dump(calibrate_LSVC,filename='calibrate_LSVC')

['calibrate_LSVC']

In [15]:
#LinearSVC
#caculate AUC in the internal validation cohort
parameter = {'loss':['squared_hinge']}
Linear_SVM_model = LinearSVC(class_weight='balanced',random_state=1,penalty = 'l2',dual=True,
                            fit_intercept=False,C=0.1)
GS1 = GridSearchCV(Linear_SVM_model, parameter,scoring='roc_auc', cv=5)
model1 = GS1.fit(X,y)
result = model1.cv_results_
auc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
#caculate 95% CI of AUC
interval1=stats.norm.interval(0.95,np.mean(auc_result),np.std(auc_result))
print('AUC:',auc_result)
print('Mean AUC:',np.mean(auc_result))
print('95% CI of AUC',interval1)

#caculate accuracy in the internal validation cohort
GS2 = GridSearchCV(Linear_SVM_model, parameter,scoring='accuracy', cv=5)
model2 = GS2.fit(X,y)
result = model2.cv_results_
acc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval2=stats.norm.interval(0.95,np.mean(acc_result),np.std(acc_result))
print('Acc',acc_result)
print('Mean Acc',np.mean(acc_result))
print('95% CI of Acc',interval2)

#caculate accuracy in the internal validation cohort
GS3 = GridSearchCV(Linear_SVM_model, parameter,scoring='f1', cv=5)
model3 = GS3.fit(X,y)
result = model3.cv_results_
f1_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval3=stats.norm.interval(0.95,np.mean(f1_result),np.std(f1_result))
print('F1_score',f1_result)
print('Mean F1',np.mean(f1_result))
print('95% CI of F1',interval3)


AUC: [0.70196078 0.7372549  0.72352941 0.7245098  0.6969697 ]
Mean AUC: 0.7168449197860963
95% CI of AUC (0.6872947692473245, 0.7463950703248682)
Acc [0.69387755 0.71428571 0.7755102  0.7755102  0.66666667]
Mean Acc 0.7251700680272108
95% CI of Acc (0.6393397681977937, 0.811000367856628)
F1_score [0.51612903 0.5625     0.62068966 0.62068966 0.52941176]
Mean F1 0.5698840214617549
95% CI of F1 (0.48335854366796327, 0.6564094992555465)


In [16]:
SVM_model = SVC(class_weight='balanced',probability=True,random_state=1,kernel='linear',
               C=2.5)
'''
parameters = {'kernel':['rbf','poly','linear','sigmod','precomputed'],'degree':[2,3,4,5,6]
,'C':[0.01,0.1,1,5,10],'gamma':[0.01,0.03, 0.05,0.07,0.09, 0.1, 0.5],'coef0':[0.0]}
step1:parameter1 = {'kernel':['rbf','poly','linear','sigmod','precomputed']}，best_params_：{'kernel': 'linear'}
step2:parameter2 = {'C':[0.01,0.05,0.1,0.5,1]}，best_params_：{'C': 2.5}
step3:parameter3 = {'gamma':['auto',0.001,0.005,0.01,0.05,0.1]}，best_params_：{'gamma': 'auto'} best_score_:0.7076

Thus,we choose LinearSVC
'''
parameter3 = {'gamma':['auto',0.001,0.005,0.01,0.05,0.1]}
GS = GridSearchCV(SVM_model, parameter3,scoring='roc_auc', cv=5)
model = GS.fit(X,y)
'''
Best_SVC_Model = SVC(class_weight='balanced',probability=True,random_state=1,
               kernel='linear',C=0.05,gamma='auto')
'''
Tuned_SVC_Model = model.best_estimator_
joblib.dump(Tuned_SVC_Model,filename='Best_SVC_Model')
print(model.best_params_)
print(model.best_score_)

{'gamma': 'auto'}
0.7076232917409389


In [17]:
result = model.cv_results_
a = len(result['split0_test_score'])
auc_result = np.concatenate([result['split0_test_score'].reshape(a,1),result['split1_test_score'].reshape(a,1),
               result['split2_test_score'].reshape(a,1),result['split3_test_score'].reshape(a,1),
               result['split4_test_score'].reshape(a,1)],axis=1)
SVM_validation_auc = auc_result[0,]
#if model.best_score_ = np.mean(LR_validation_auc),we obtain the correct AUC in the validation cohort
print(model.best_score_)
print(np.mean(SVM_validation_auc))
SVM_validation_auc

0.7076232917409389
0.7076232917409389


array([0.67843137, 0.70980392, 0.70980392, 0.71078431, 0.72929293])

In [18]:
RDF_model = RandomForestClassifier(class_weight="balanced",random_state=1,n_estimators=50,
                                  criterion='gini',max_depth=12,min_samples_leaf=5,
                                  min_samples_split=2)
'''
parameters = {'criterion':['gini','entropy'],'n_estimators':range(1,100),'max_depth':range(1,100),
,'min_samples_split':range(2,10),'min_samples_leaf':range(1,10),'max_features':['sqrt',3,4,5,6,7]}
step1:parameter1 = {'n_estimators':range(1,100)} best_params_:{'n_estimators':50}
step2:parameter2 = {'criterion':['gini','entropy']} best_params_:{'criterion': 'gini'}
step3:parameter3 = {'max_depth':range(1,100)} best_params_:{'max_depth': 12}
step4:parameter4 = {'min_samples_split':range(2,10),'min_samples_leaf':range(1,10)} best_params_:{'min_samples_leaf': 5, 'min_samples_split': 2}
step5:parameter5 = {'max_features':['sqrt','log2',2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]} 
best_params_:{'max_features': 3}
Thus, the best RDF model:Best_RDF_Model =  RandomForestClassifier(class_weight="balanced",random_state=1,n_estimators=70
                                  ,criterion='gini',max_depth=5,min_samples_split=2,
                                  min_samples_leaf=5,max_features=3)

'''
parameter5 = {'max_features':['sqrt','log2',2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]} 
GS = GridSearchCV(RDF_model, parameter5,scoring='roc_auc', cv=5)
model = GS.fit(X,y)
'''
Best_RDF_Model = RandomForestClassifier(class_weight="balanced",random_state=1,n_estimators=70
                                        ,criterion='gini',max_depth=5,min_samples_split=2,
                                        min_samples_leaf=5,max_features='sqrt')
'''
Tuned_RDF_Model = model.best_estimator_
joblib.dump(Tuned_RDF_Model,filename='Best_RDF_Model')                                
print(model.best_params_)
print(model.best_score_)

{'max_features': 3}
0.748871063576946


In [19]:
#calibration for Random forest
isotonic = CalibratedClassifierCV(model.best_estimator_, cv=5, method='isotonic')
calibrate_RDF = isotonic.fit(X,y)
joblib.dump(calibrate_RDF,filename='calibrate_RDF')

['calibrate_RDF']

In [20]:
#RDF
#caculate AUC in the internal validation cohort
parameter = {'max_features':[3]}
GS1 = GridSearchCV(RDF_model, parameter,scoring='roc_auc', cv=5)
model1 = GS1.fit(X,y)
result = model1.cv_results_
auc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
#caculate 95% CI of AUC
interval1=stats.norm.interval(0.95,np.mean(auc_result),np.std(auc_result))
print('AUC:',auc_result)
print('Mean AUC:',np.mean(auc_result))
print('95% CI of AUC',interval1)

#caculate accuracy in the internal validation cohort
GS2 = GridSearchCV(RDF_model, parameter,scoring='accuracy', cv=5)
model2 = GS2.fit(X,y)
result = model2.cv_results_
acc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval2=stats.norm.interval(0.95,np.mean(acc_result),np.std(acc_result))
print('Acc',acc_result)
print('Mean Acc',np.mean(acc_result))
print('95% CI of Acc',interval2)

#caculate accuracy in the internal validation cohort
GS3 = GridSearchCV(RDF_model, parameter,scoring='f1', cv=5)
model3 = GS3.fit(X,y)
result = model3.cv_results_
f1_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval3=stats.norm.interval(0.95,np.mean(f1_result),np.std(f1_result))
print('F1_score',f1_result)
print('Mean F1',np.mean(f1_result))
print('95% CI of F1',interval3)

AUC: [0.70196078 0.81764706 0.69215686 0.7245098  0.80808081]
Mean AUC: 0.748871063576946
95% CI of AUC (0.6442505535790908, 0.8534915735748012)
Acc [0.69387755 0.79591837 0.75510204 0.67346939 0.75      ]
Mean Acc 0.7336734693877551
95% CI of Acc (0.6468649268875631, 0.8204820118879471)
F1_score [0.44444444 0.5        0.5        0.38461538 0.57142857]
Mean F1 0.48009768009768006
95% CI of F1 (0.3576303272929534, 0.6025650329024067)


In [21]:
GDB_model = GradientBoostingClassifier(random_state=1,n_estimators=31,min_samples_leaf=7,
                                      min_samples_split=2,max_features='sqrt',max_depth=13)
'''
parameters ={'learning_rate':[0.001,0.005,0.01,0.05,0.1,0.5,1]
                ,'n_estimators':range(1,100)
                ,'max_depth':range(1,100),'min_samples_split':[2,3,4,5,6,7,8,9,10]
                ,'max_features':[3,4,5,6,7],
                ,'subsample':[0.6,0.7,0.75,0.8,0.85,0.9,0.95,1]
                ,'criterion':['friedman_mse', 'mse', 'mae']
                }
step1:parameter1 = {'n_estimators':range(1,100)} best_params_:{'n_estimators': 31}
step2:parameter2 = {'learning_rate':[0.001,0.005,0.01,0.05,0.1,0.5,1]} best_params_:{'learning_rate': 0.1}
step3:parameter3 = {'min_samples_split':range(2,10),'min_samples_leaf':range(1,10)} best_params_:{'min_samples_leaf': 7, 'min_samples_split': 2}
step4:parameter4 = {'max_features':['sqrt','log2',2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]} best_params_:{'max_features': 'sqrt'}
step5:parameter5 = {'max_depth':range(1,100)} best_params_:{'max_depth': 13}
step6:parameter6 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9,0.95,1]} best_params_:{'subsample': 1}
'''
parameters = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9,0.95,1]} 
GS = GridSearchCV(GDB_model, parameters,scoring='roc_auc', cv=5)
model = GS.fit(X,y)
'''
Best_GDB_Model =  GradientBoostingClassifier(random_state=1,n_estimators=30,learning_rate=0.1,
                                      max_features=3,max_depth=2,min_samples_split=3,
                                      subsample=0.7)
'''
Tuned_GDB_Model = model.best_estimator_
joblib.dump(Tuned_GDB_Model,filename='Best_GDB_Model') 
print(model.best_params_)
print(model.best_score_)


{'subsample': 1}
0.7374866310160428


In [22]:
#calibration for Random forest
isotonic = CalibratedClassifierCV(model.best_estimator_, cv=5, method='isotonic')
calibrate_GDB = isotonic.fit(X,y)
joblib.dump(calibrate_GDB,filename='calibrate_GDB')

['calibrate_GDB']

In [23]:
#GDB
#caculate AUC in the internal validation cohort
parameter = {'subsample':[1]}
GS1 = GridSearchCV(GDB_model, parameter,scoring='roc_auc', cv=5)
model1 = GS1.fit(X,y)
result = model1.cv_results_
auc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
#caculate 95% CI of AUC
interval1=stats.norm.interval(0.95,np.mean(auc_result),np.std(auc_result))
print('AUC:',auc_result)
print('Mean AUC:',np.mean(auc_result))
print('95% CI of AUC',interval1)

#caculate accuracy in the internal validation cohort
GS2 = GridSearchCV(GDB_model, parameter,scoring='accuracy', cv=5)
model2 = GS2.fit(X,y)
result = model2.cv_results_
acc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval2=stats.norm.interval(0.95,np.mean(acc_result),np.std(acc_result))
print('Acc',acc_result)
print('Mean Acc',np.mean(acc_result))
print('95% CI of Acc',interval2)

#caculate accuracy in the internal validation cohort
GS3 = GridSearchCV(GDB_model, parameter,scoring='f1', cv=5)
model3 = GS3.fit(X,y)
result = model3.cv_results_
f1_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval3=stats.norm.interval(0.95,np.mean(f1_result),np.std(f1_result))
print('F1_score',f1_result)
print('Mean F1',np.mean(f1_result))
print('95% CI of F1',interval3)

AUC: [0.65098039 0.8        0.72156863 0.7754902  0.73939394]
Mean AUC: 0.7374866310160428
95% CI of AUC (0.6371926012433831, 0.8377806607887026)
Acc [0.67346939 0.71428571 0.75510204 0.81632653 0.6875    ]
Mean Acc 0.7293367346938775
95% CI of Acc (0.6281407681640724, 0.8305327012236826)
F1_score [0.33333333 0.41666667 0.33333333 0.66666667 0.51612903]
Mean F1 0.4532258064516129
95% CI of F1 (0.20593023311449316, 0.7005213797887326)


In [24]:
import tensorflow as tf
np.random.seed(2)
np.random.seed(2)
tf.random.set_seed(2)

X1 = X.values
y1 = y.values
def bliud_model(neuron1=8,neuron2=4,lr=1,momentums=0.7):
    model = models.Sequential()
    model.add(layers.Dense(neuron1,activation='relu',input_shape=(24,)))
    #model.add(layers.Dropout(dropout))
    model.add(layers.Dense(neuron2,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    sgd = SGD(learning_rate=lr,momentum=momentums)
    model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])
    return model
'''
 parameters ={'neurons':range(6,20)，'neuron2':[3,4,5,6,7,8,9,10]
                 'lr':[0.1,0.2,0.3,0.4,0.5],'momentum':[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
               ,'batch_size':[5,10,15,20,25] ,'nb_epoch':[16,20,24,28,32],
               ,'weight_constraint':[1,2,3,4,5],'dropout':[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
                'activation':['softmax','softplus','softsign','relu','tanh','sigmoid','hard_sigmoid','linear']}
one layer neuron
Beacause of small samples of our dataset and good performance for activation relu,we didn't perform hyperparameter optimization for
dropout, weight_constraint, and activation.
step1:parameter1 = {'neurons':range(6:20)} best_params_:{'neurons': 15}
step2:parameter2 = {'batch_size':range(4,33),'nb_epoch':range(10,30)}  best_params_:{'batch_size': 28, 'nb_epoch': 15}
step3:parameter3 = {'lr':[0.001,0.01,0.05,0.1,0.5,1,2.5,5],'momentums':[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
best_params_: {'lr': 1, 'momentums': 0.8}  best_score_:0.688

Two layer neuron
Because of small sample size,The maximum number of layers of the neural network is set to 2
step1:parameter1 = {'neuron1':range(6,20),'neuron2':range(3,15)} best_params_:{'neuron1': 8, 'neuron2': 4}
step2:parameter2 = {'lr':[0.001,0.01,0.05,0.1,0.5,1,2.5,5],'momentums':[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
best_params_：{'lr': 1, 'momentums': 0.7} best_score_: 0.63029
step3:parameter3 = {'batch_size':range(4,65,4)} best_params_:{'batch_size': 32}
step4:parameter4 = {'nb_epoch':range(10,30)} best_params_:{'nb_epoch': 22} model.best_score_: 0.657

Above all,the MLP_model achieved best performance when the number of layer neuron is set to 1,
The optimal parameter is {'neurons': 15},{'batch_size': 28, 'nb_epoch': 15},{'lr': 1, 'momentums': 0.8}
'''
parameter4 = {'nb_epoch':range(10,30)}
MLP_model = KerasClassifier(build_fn=bliud_model,batch_size=32)
GS = GridSearchCV(estimator=MLP_model, param_grid=parameter4,scoring='roc_auc', cv=5)
model = GS.fit(X,y)

print(model.best_params_)
print(model.best_score_)

8/8 [==============================] - 0s 1ms/step - loss: 0.6480 - accuracy: 0.6352
{'nb_epoch': 23}
0.6523836767346276


In [25]:
import tensorflow as tf
np.random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

def bliud_model1(neuron1=15,lr=1,momentums=0.8):
    model = models.Sequential()
    model.add(layers.Dense(neuron1,activation='relu',input_shape=(24,)))
    model.add(layers.Dense(1,activation='sigmoid'))
    sgd = SGD(learning_rate=lr,momentum=momentums)
    model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])
    return model
parameter = {'lr':[1],'momentums':[0.8]}
MLP_model = KerasClassifier(build_fn=bliud_model1,batch_size=28,nb_epoch=15)
GS1 = GridSearchCV(estimator=MLP_model, param_grid=parameter,scoring='roc_auc', cv=5)
model1 = GS1.fit(X,y)
result = model1.cv_results_
auc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
#caculate 95% CI of AUC
interval1=stats.norm.interval(0.95,np.mean(auc_result),np.std(auc_result))
Tuned_MLP_Model = model1.best_estimator_
Tuned_MLP_Model.model1.save('Best_MLP_Model.h5')

#caculate accuracy in the internal validation cohort
GS2 = GridSearchCV(MLP_model, parameter,scoring='accuracy', cv=5)
model2 = GS2.fit(X,y)
result = model2.cv_results_
acc_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval2=stats.norm.interval(0.95,np.mean(acc_result),np.std(acc_result))


#caculate accuracy in the internal validation cohort
GS3 = GridSearchCV(MLP_model, parameter,scoring='f1', cv=5)
model3 = GS3.fit(X,y)
result = model3.cv_results_
f1_result = np.concatenate([result['split0_test_score'],result['split1_test_score'],
               result['split2_test_score'],result['split3_test_score'],
               result['split4_test_score']],axis=0)
interval3=stats.norm.interval(0.95,np.mean(f1_result),np.std(f1_result))
print('AUC:',auc_result)
print('Mean AUC:',np.mean(auc_result))
print('95% CI of AUC',interval1)

print('Acc',acc_result)
print('Mean Acc',np.mean(acc_result))
print('95% CI of Acc',interval2)

print('F1_score',f1_result)
print('Mean F1',np.mean(f1_result))
print('95% CI of F1',interval3)

7/7 [==============================] - 0s 2ms/step - loss: 0.6557 - accuracy: 0.6821


C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


7/7 [==============================] - 0s 2ms/step - loss: 0.6750 - accuracy: 0.6308


C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


7/7 [==============================] - 0s 0s/step - loss: 0.6579 - accuracy: 0.6462


...

7/7 [==============================] - 0s 2ms/step - loss: 0.6291 - accuracy: 0.7179


C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


7/7 [==============================] - 0s 2ms/step - loss: 0.6808 - accuracy: 0.6224


C:\Users\lenovo\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


9/9 [==============================] - 0s 1ms/step - loss: 0.6556 - accuracy: 0.6803
AUC: [0.65510204 0.53787879 0.72972973 0.69912281 0.69327731]
Mean AUC: 0.6630221352733516
95% CI of AUC (0.5318593227866524, 0.7941849477600509)
Acc [0.71428571 0.67346939 0.75510204 0.6122449  0.72916667]
Mean Acc 0.6968537414965985
95% CI of Acc (0.599066082027547, 0.7946414009656501)
F1_score [0.33333333 0.         0.         0.18181818 0.        ]
Mean F1 0.10303030303030303
95% CI of F1 (-0.16151800304369562, 0.3675786091043017)


In [26]:
##calibration for Random forest
import tensorflow as tf
np.random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

def calibration_model():
    model = models.Sequential()
    model.add(layers.Dense(15,activation='relu',input_shape=(24,)))
    model.add(layers.Dense(1,activation='sigmoid'))
    sgd = SGD(learning_rate=1,momentum=0.8)
    model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])
    return model
MLP_model = KerasClassifier(calibration_model,batch_size=28,nb_epoch=15)
isotonic = CalibratedClassifierCV(MLP_model, cv=5, method='isotonic')
#calibrate_MLP = isotonic.fit(X,y)
